In [5]:
import pandas as pd
import keras
from keras.models import load_model
import numpy as np
from tqdm import tqdm

In [6]:
model_path = './models/basic_regression_model_v3_float32.h5'

df = pd.read_csv('./data/csv/test_acc10_200.csv')
model = load_model(model_path)

In [7]:
df.head()

,lig_id,pro_id,dest,score
0,2701,2701,../data/processed_data/test_acc10/2701_pro_270...,1
1,2701,2702,../data/processed_data/test_acc10/2702_pro_270...,0
2,2701,2703,../data/processed_data/test_acc10/2703_pro_270...,0
3,2701,2704,../data/processed_data/test_acc10/2704_pro_270...,0
4,2701,2705,../data/processed_data/test_acc10/2705_pro_270...,0


In [8]:
# group by ligand
# for each group, compute top10 accuracy manually

matches = 0
total = float(len(df.lig_id.unique()))
debug = False

for lig_id, grp in tqdm(df.groupby('lig_id')):
    grp.reset_index(inplace=True)
    
    match_id = grp.index[grp['pro_id']==lig_id]
    
    # load all features
    n_channels = 16
    dims = (24,24,24)
    X = np.empty((len(grp.index), *dims, n_channels))
    for row in grp.itertuples():
        X[row[0],] = np.load(row.dest)
        
    probabilities = model.predict_on_batch(X)
    probabilities = probabilities.flatten()
    
    largest_first = reversed(np.argsort(probabilities).tolist())
    top10 = list(largest_first)[:10]
    
    if debug:
        print('ligid: ', lig_id)
        print('match_id: ', match_id)
        print('top_10: ', top10)
        print()
    
    if match_id in top10:
        matches += 1
        
print(matches/total)    

  0%|          | 0/200 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed_data/test_acc10/2701_pro_2701_lig.npy'